In [2]:
import retro
import random

from simulator import genesis as genesis

In [3]:
# in Jupyter env might have been created already
if 'env' not in locals():
    env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', state="Champion.Level1.RyuVsGuile")
    # env = retro.make(game='MortalKombat3-Genesis')

agent = genesis.Agent(env)

def take_action(actions):
    return random.choice(actions)

In [4]:
from time import sleep
obs = env.reset()
while True:
    ack = take_action(agent.actions)      

    state, reward, done, info = env.step(ack)
    env.render()
    if done or info['enemy_matches_won'] > 0:
        obs = env.reset()
    sleep(0.01)
env.close()

KeyboardInterrupt: 

In [ ]:
# bina = agent.binarize_state(state)
# small = agent.resize_state(bina)

In [ ]:
bina.shape, small.shape